# 运行模型

In [1]:
import sys
import gc, csv
import numpy as np
import pandas as pd
sys.path.append("./global")
from helper import Timer, AnalysisCSV, ReadCSV, OFFLINE, CalQAuc, ORI_TRAIN_NAMES, ReduceMemUsage
from stage2.code.feature_selection import select_features
print(OFFLINE)

False


## 拼接特征

In [2]:
combine_all_features = True

In [3]:
base_feature_dir = "./stage1/output/"
base_npy_dir = "./stage2/input/"
if OFFLINE:
    csv_train_concat_feature_file = base_feature_dir + "debug_train_concat_feature.csv"
    npy_train_concat_feature_file = base_npy_dir + "debug_train_concat_feature.npy"
    npy_label_file = base_npy_dir + "debug_train_labels.npy"
    npy_train_auc_file = base_npy_dir + "debug_train_for_cal_auc.npy"
    train_file = "./stage1/input/train_last_50w.csv"
    feature_names = "./stage2/input/debug_feature_names.csv"
else:
    csv_train_concat_feature_file = base_feature_dir + "online_train_concat_feature.csv"
    csv_test_concat_feature_file = base_feature_dir + "online_test_concat_feature.csv"
    npy_train_concat_feature_file = base_npy_dir + "online_train_concat_feature.npy"
    npy_test_concat_feature_file = base_npy_dir + "online_test_concat_feature.npy"
    npy_train_auc_file = base_npy_dir + "online_train_for_cal_auc.npy"
    npy_label_file = base_npy_dir + "online_train_labels.npy"
    train_file = "./stage1/input/train_last_1000w.csv"
    feature_names = "./stage2/input/online_feature_names.csv"

### 修改合并文件列表

In [4]:
if OFFLINE:
    train_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/debug_trainTextMining_len.csv.gz",
        "text_mining_feature/debug_trainTextMining_lenRel.csv.gz",
        "text_mining_feature/debug_trainTextMining_editSim.csv.gz",
        "text_mining_feature/debug_trainTextMining_ngramSimV1.csv.gz",
        "word2vec/debug_trainWord2vec_senVecavgSimV1.csv.gz",
        # "word2vec/debug_trainWord2vec_senVectfidfSim.csv.gz",
        "magic/debug_trainMagic_queryFreq.csv.gz",
        "magic/debug_trainMagic_ctrRateLocal.csv.gz",
        "magic/debug_trainMagic_diffSetSim.csv.gz",
        "magic/debug_trainMagic_diffSetLen.csv.gz",
        # "vector_space/debug_trainVectorSpace_tfidfVecSim.csv.gz",
        # "vector_space/debug_trainVectorSpace_tfidfVecLen.csv.gz",
        "vector_space/debug_trainVectorSpace_tfidfVecLenV1.csv.gz",
        "vector_space/debug_trainVectorSpace_tfidfVecSimV1.csv.gz",
        "vector_space/debug_trainVectorSpace_tfidfVecLenMean.csv.gz"
        # "vector_space/debug_trainVectorSpace_tfidfVecLenChar.csv.gz",
        # "text_mining_feature/debug_trainTextMining_ngramSubSeqV1.csv.gz"
        # "magic/debug_trainMagic_diffSetTfidfLen.csv.gz"
        # "magic/debug_trainMagic_diffSetSimChars.csv.gz",
        # "magic/debug_trainMagic_diffSetLenChars.csv.gz",
        # "word2vec/debug_trainWord2vec_senVectfidfSimV1.csv.gz",
        # "word2vec/debug_trainWord2vec_senVectfidfSimV1Sum.csv.gz"
        # "vector_space/debug_trainVectorSpace_tfidfVecLenMore.csv.gz"
        # "vector_space/debug_trainVectorSpace_tfidfVecLenMulti.csv.gz"
        # "vector_space/debug_trainVectorSpace_ngramSimHash.csv.gz", # 55.59
        # "vector_space/debug_trainVectorSpace_ngramSimHashV1.csv.gz"
    ]]
else:
    train_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/online_trainTextMining_len.csv.gz",
        "text_mining_feature/online_trainTextMining_lenRel.csv.gz",
        "text_mining_feature/online_trainTextMining_editSim.csv.gz",
        "text_mining_feature/online_trainTextMining_ngramSimV1.csv.gz",
        "word2vec/online_trainWord2vec_senVecavgSim.csv.gz",
        "magic/online_trainMagic_queryFreq.csv.gz",
        "magic/online_trainMagic_ctrRateLocal.csv.gz",
        "magic/online_trainMagic_diffSetSim.csv.gz",
        "magic/online_trainMagic_diffSetLen.csv.gz",
        "vector_space/online_trainVectorSpace_tfidfVecSim.csv.gz",
        "vector_space/online_trainVectorSpace_tfidfVecLenV2.csv.gz"
    ]]
    test_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/online_testTextMining_len.csv.gz",
        "text_mining_feature/online_testTextMining_lenRel.csv.gz",
        "text_mining_feature/online_testTextMining_editSim.csv.gz",
        "text_mining_feature/online_testTextMining_ngramSimV1.csv.gz",
        "word2vec/online_testWord2vec_senVecavgSim.csv.gz",
        "magic/online_testMagic_queryFreq.csv.gz",
        "magic/online_testMagic_ctrRateLocal.csv.gz",
        "magic/online_testMagic_diffSetSim.csv.gz",
        "magic/online_testMagic_diffSetLen.csv.gz",
        "vector_space/online_testVectorSpace_tfidfVecSim.csv.gz",
        "vector_space/online_testVectorSpace_tfidfVecLenV2.csv.gz"
    ]]

In [5]:
# 合并所有特征
def CombineFeatures(feature_files, savefile, y=None, select_feature=False, threshold=0.9):
    global feature_names
    # 拼接特征
    with Timer("concat features"):
        tmp = [ReduceMemUsage(ReadCSV(f, iterator=False)) for f in feature_files]
        features = pd.concat(tmp, axis=1)
        for _ in tmp:
            print("feature file shape:", _.shape)
            del _
        gc.collect()
    # change inf to nan
    with Timer("Change inf to nan"):
        features = features.astype(dtype=np.float32)
        features = features.replace(np.inf, np.nan)
    # fill nan
    with Timer("Fill nan"):
        features = features.fillna(0.0)
    print("total feature shape:", features.shape)
    
    # # 特征筛选
    # cols_name = features.columns.values
    # if select_feature:
    #     with Timer("select features"):
    #         y = np.load(y)
    #         select_features("simple", features.values, y, cols_name, "./stage2/output/feature_selection/")
    #         select_features("lgb", features.values, y, cols_name, "./stage2/output/feature_selection/")
    #         select_features("rf", features.values, y, cols_name, "./stage2/output/feature_selection/")
    # # 更新特征
    # csvfiles = [
    #     "features_weight_lgb.csv",
    #     "features_weight_rf.csv",
    #     "features_weight_simple.csv"
    # ]
    # csvfiles = ["./stage2/output/feature_selection/"+_ for _ in csvfiles]
    # feature_imports = pd.concat([ReadCSV(_, iterator=False) for _ in csvfiles], axis=1)
    # # print(feature_imports[:2])
    # import_score = feature_imports.mean(axis=1)
    # print("origin feature score:", import_score)
    # min_score = import_score.nlargest(int(import_score.shape[0]*threshold)).min()
    # print("drop feature score less than:", min_score)
    # # 筛选特征
    # features.drop(cols_name[import_score < min_score], axis=1, inplace=True, errors='ignore')
    
    # 存储header
    with open(feature_names, "w") as f:
        f.write(",".join(list(features.columns.values)))
    # 存储特征
    tmp1 = np.array(features)
    
    print("features shape:", tmp1.shape)
    print("two rows of features:")
    print(tmp1[:2])
    
    np.save(savefile, tmp1)
    print("feature file saved to "+savefile)
    del features, tmp1
    gc.collect()

### 拼接

In [6]:
# 拼接特征
if combine_all_features:
    CombineFeatures(train_feature_files, npy_train_concat_feature_file, npy_label_file, True, 0.9)
    if not OFFLINE:
        CombineFeatures(test_feature_files, npy_test_concat_feature_file)

----->Started 'concat features' block...
Mem. usage decreased to 143.05 Mb (76.6% reduction)
Mem. usage decreased to 143.05 Mb (76.6% reduction)
Mem. usage decreased to 104.90 Mb (82.8% reduction)
Mem. usage decreased to 305.18 Mb (75.0% reduction)
Mem. usage decreased to 133.51 Mb (75.0% reduction)
Mem. usage decreased to  9.54 Mb (87.5% reduction)
Mem. usage decreased to 19.07 Mb (75.0% reduction)
Mem. usage decreased to 228.88 Mb (75.0% reduction)
Mem. usage decreased to 171.66 Mb (67.9% reduction)
Mem. usage decreased to 57.22 Mb (75.0% reduction)
Mem. usage decreased to 190.73 Mb (72.2% reduction)
feature file shape: (10000000, 8)
feature file shape: (10000000, 8)
feature file shape: (10000000, 8)
feature file shape: (10000000, 16)
feature file shape: (10000000, 7)
feature file shape: (10000000, 1)
feature file shape: (10000000, 1)
feature file shape: (10000000, 12)
feature file shape: (10000000, 7)
feature file shape: (10000000, 3)
feature file shape: (10000000, 9)
----->Finished

In [8]:
# tmp = ReadCSV("./stage2/output/feature_selection/features_weight_rf.csv", iterator=False)
# print(tmp)

In [9]:
!cat stage2/input/online_feature_names.csv

online_testTextMining_text_q-len,online_testTextMining_text_t-len,online_testTextMining_text_qt-len-radio,online_testTextMining_text_tq-len-radio,online_testTextMining_unigrams_q-len,online_testTextMining_unigrams_t-len,online_testTextMining_unigrams_qt-len-radio,online_testTextMining_unigrams_tq-len-radio,online_testTextMining_unichars_qt-diff,online_testTextMining_unichars_qt-max,online_testTextMining_unichars_qt-min,online_testTextMining_unichars_qt-avg,online_testTextMining_unigrams_qt-diff,online_testTextMining_unigrams_qt-max,online_testTextMining_unigrams_qt-min,online_testTextMining_unigrams_qt-avg,online_testTextMining_text_distance,online_testTextMining_text_jaro,online_testTextMining_text_jaro_winkler,online_testTextMining_text_ratio,online_testTextMining_text_partial_ratio,online_testTextMining_text_token_sort_ratio,online_testTextMining_text_partial_token_sort_ratio,online_testTextMining_text_token_set_ratio,online_testTextMining_unichars_dice_ratio,online_testTextMining_u

In [10]:
!cat stage2/input/debug_feature_names.csv

debug_trainTextMining_text_q-len,debug_trainTextMining_text_t-len,debug_trainTextMining_text_qt-len-radio,debug_trainTextMining_text_tq-len-radio,debug_trainTextMining_unigrams_q-len,debug_trainTextMining_unigrams_t-len,debug_trainTextMining_unigrams_qt-len-radio,debug_trainTextMining_unigrams_tq-len-radio,debug_trainTextMining_unichars_qt-diff,debug_trainTextMining_unichars_qt-max,debug_trainTextMining_unichars_qt-min,debug_trainTextMining_unichars_qt-avg,debug_trainTextMining_unigrams_qt-diff,debug_trainTextMining_unigrams_qt-max,debug_trainTextMining_unigrams_qt-min,debug_trainTextMining_unigrams_qt-avg,debug_trainTextMining_text_distance,debug_trainTextMining_text_jaro,debug_trainTextMining_text_jaro_winkler,debug_trainTextMining_text_ratio,debug_trainTextMining_text_partial_ratio,debug_trainTextMining_text_token_sort_ratio,debug_trainTextMining_text_partial_token_sort_ratio,debug_trainTextMining_text_token_set_ratio,debug_trainTextMining_unichars_dice_ratio,debug_trainTextMining_u

In [11]:
tmp = np.load("/home/kesci/work/stage2/output/train/lgb_gbdt_1561381624_auc0.667957_loss0.533527_std0.003079.npy")
print(tmp[:10])

[0.22576493 0.19011801 0.21707552 0.18630755 0.21659341 0.20506354
 0.20058367 0.1962723  0.21583764 0.20020342]


## 运行模型

In [12]:
! pip install  --index "http://pypi/simple" --trusted-host pypi  hyperopt
# ! pip install  --index "http://pypi/simple" --trusted-host pypi  pyemd

    100% |████████████████████████████████| 122kB 79.3MB/s ta 0:00:01
    100% |████████████████████████████████| 419kB 74.9MB/s ta 0:00:01


In [7]:
from stage2.code.run_hyperopt import LoadDataset, run_lgb_gbdt, run_lr, run_lgb_gbdt_fast, run_lgb_dart

In [8]:
# 加载数据集
if OFFLINE:
    npy_test_concat_feature_file = None
LoadDataset(in_train_feature=npy_train_concat_feature_file, 
            in_train_label=npy_label_file, 
            in_test_feature=npy_test_concat_feature_file,
            in_q_auc_cal=npy_train_auc_file)

Feature names: ['online_testTextMining_text_q-len', 'online_testTextMining_text_t-len', 'online_testTextMining_text_qt-len-radio', 'online_testTextMining_text_tq-len-radio', 'online_testTextMining_unigrams_q-len', 'online_testTextMining_unigrams_t-len', 'online_testTextMining_unigrams_qt-len-radio', 'online_testTextMining_unigrams_tq-len-radio', 'online_testTextMining_unichars_qt-diff', 'online_testTextMining_unichars_qt-max', 'online_testTextMining_unichars_qt-min', 'online_testTextMining_unichars_qt-avg', 'online_testTextMining_unigrams_qt-diff', 'online_testTextMining_unigrams_qt-max', 'online_testTextMining_unigrams_qt-min', 'online_testTextMining_unigrams_qt-avg', 'online_testTextMining_text_distance', 'online_testTextMining_text_jaro', 'online_testTextMining_text_jaro_winkler', 'online_testTextMining_text_ratio', 'online_testTextMining_text_partial_ratio', 'online_testTextMining_text_token_sort_ratio', 'online_testTextMining_text_partial_token_sort_ratio', 'online_testTextMining_

In [15]:
print(npy_train_concat_feature_file)
print(npy_label_file)
print(npy_test_concat_feature_file)
print(npy_train_auc_file)

./stage2/input/debug_train_concat_feature.npy
./stage2/input/debug_train_labels.npy
None
./stage2/input/debug_train_for_cal_auc.npy


In [26]:
run_lgb_gbdt("last_version")

{                                                   
 "bagging_fraction": 1.0,
 "bagging_freq": 6.0,
 "feature_fraction": 0.7000000000000001,
 "learning_rate": 0.14,
 "max_bin": 511,
 "min_data_in_leaf": 10.0,
 "num_iterations": 250.0,
 "num_leaves": 45.0,
 "num_threads": 8
}
[50]	valid's binary_logloss: 0.529879               
[100]	valid's binary_logloss: 0.528973              
[150]	valid's binary_logloss: 0.528668              
[200]	valid's binary_logloss: 0.528551              
[250]	valid's binary_logloss: 0.528506              
[50]	valid's binary_logloss: 0.530041               
[100]	valid's binary_logloss: 0.529108              
[150]	valid's binary_logloss: 0.528802              
[200]	valid's binary_logloss: 0.528656              
[250]	valid's binary_logloss: 0.528599              
[50]	valid's binary_logloss: 0.529719               
[100]	valid's binary_logloss: 0.528843              
[150]	valid's binary_logloss: 0.52854               
[200]	valid's binary_logloss: 0.52